In [13]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/train'
validation_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/valid'

batch_size = 1000
img_size = 32

train_datagen = ImageDataGenerator()

# train_datagen = ImageDataGenerator(rescale=1/255,
#                                    rotation_range=20,  # 지정된 각도 범위내에서 임의로 원본 이미지를 회전
#                                    width_shift_range=0.1,
#                                    height_shift_range=0.1,
#                                    zoom_range=0.1,    #  1-수치 혹은 1+수치만큼 확대 및 축소
#                                    )



validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size,img_size),                      
    batch_size=batch_size,     
    class_mode='categorical'                    
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_size,img_size),                       
    batch_size=batch_size,    
    class_mode='categorical'                    
)

Found 54332 images belonging to 2 classes.
Found 13585 images belonging to 2 classes.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0

np.random.seed(42)

# gpu 메모리 최대로 잡는 것을 방지
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# 모델 구축

# EfficientNetB0 모델 불러오기
pre_trained_model = EfficientNetB0(include_top=False, weights='imagenet',
                                  input_shape=(img_size, img_size, 3))
pre_trained_model.trainable = False
pre_trained_model.summary()

model = Sequential()
model.add(pre_trained_model)
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))

model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy',
                metrics=['accuracy'])

1 Physical GPUs, 1 Logical GPUs
Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 32, 32, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 32, 32, 3)    7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 33, 33, 3)    0           normalization[0][0]              
_____________________________________________________

In [14]:
early_stopping = EarlyStopping(patience = 15) # 조기종료 콜백함수 정의

# 데이터 개수 / batch_size
steps_per_epoch = 54332 / batch_size
val_steps = 13585 / batch_size

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    callbacks=[early_stopping],
                    verbose=1
                    )

Epoch 1/100
55/54 [==============================] - 18s 331ms/step - loss: 0.0798 - accuracy: 0.9710 - val_loss: 0.0705 - val_accuracy: 0.9751
Epoch 2/100
55/54 [==============================] - 18s 327ms/step - loss: 0.0794 - accuracy: 0.9706 - val_loss: 0.0713 - val_accuracy: 0.9745
Epoch 3/100
55/54 [==============================] - 18s 331ms/step - loss: 0.0790 - accuracy: 0.9712 - val_loss: 0.0708 - val_accuracy: 0.9745
Epoch 4/100
55/54 [==============================] - 19s 341ms/step - loss: 0.0787 - accuracy: 0.9710 - val_loss: 0.0727 - val_accuracy: 0.9736
Epoch 5/100
55/54 [==============================] - 18s 330ms/step - loss: 0.0780 - accuracy: 0.9719 - val_loss: 0.0704 - val_accuracy: 0.9745
Epoch 6/100
55/54 [==============================] - 18s 333ms/step - loss: 0.0777 - accuracy: 0.9711 - val_loss: 0.0702 - val_accuracy: 0.9749
Epoch 7/100
55/54 [==============================] - 18s 332ms/step - loss: 0.0797 - accuracy: 0.9708 - val_loss: 0.0698 - val_accuracy:

In [15]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/test'

batch_size = 50
img_size = 32

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size,img_size),                      
    batch_size=batch_size,     
    class_mode='categorical'                    
)

Found 16981 images belonging to 2 classes.


In [16]:
# 모델 평가

steps = 16981 / batch_size

print("-- Evaluate --")
scores = model.evaluate(test_generator, steps=steps)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
340/339 [==============================] - 5s 15ms/step - loss: 0.0584 - accuracy: 0.9796
accuracy: 97.96%


In [66]:
model.save('C:/Users/s_csmscox/jupyterSave/eye_blink/eye_blink_EfficientNetB0.h5')

In [56]:
import pathlib
import os,glob
import numpy as np
from PIL import Image
import cv2

from tensorflow.keras.utils import to_categorical

# 이미지를 numpy 배열로 만들기

np.random.seed(42)

directory = 'C:/Users/s_csmscox/jupyterSave/eye_blink/test'
categories = ["0","1"] # dog = 1, cat = 0
nb_classes = len(categories)

w = 32
h = 32

X = []
y = []

for idx, obj in enumerate(categories):

    image_dir = directory + "/" + obj
    files = glob.glob(image_dir+"/*.png")
    print(obj, " 파일 길이 : ", len(files))
    
    for i, f in enumerate(files):
        img = cv2.imread(f, cv2.IMREAD_COLOR)
        
        # 사이즈 조절
        img = cv2.resize(img, (w,h))
        
        X.append(img)
        y.append(idx)

        if i % 10000 == 0:
            print(obj, " : ", f)

0  파일 길이 :  8390
0  :  C:/Users/s_csmscox/jupyterSave/eye_blink/test/0\s0001_00002_0_0_0_0_0_01.png
1  파일 길이 :  8591
1  :  C:/Users/s_csmscox/jupyterSave/eye_blink/test/1\s0001_01842_0_0_1_0_0_01.png


In [57]:
X = np.array(X)
y = np.array(y)

In [60]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X)
y_pred = np.argmax(y_pred, axis=-1)

array([[9.9597198e-01, 4.0279781e-03],
       [8.8347578e-01, 1.1652415e-01],
       [9.9495310e-01, 5.0468910e-03],
       ...,
       [2.0240283e-05, 9.9997973e-01],
       [8.6710453e-03, 9.9132895e-01],
       [4.3316380e-04, 9.9956685e-01]], dtype=float32)

In [65]:
target_names = ['close', 'open']
print(classification_report(y, y_pred, target_names=target_names, digits=4))

              precision    recall  f1-score   support

       close     0.9771    0.9810    0.9791      8390
        open     0.9814    0.9775    0.9795      8591

    accuracy                         0.9793     16981
   macro avg     0.9793    0.9793    0.9793     16981
weighted avg     0.9793    0.9793    0.9793     16981



In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

early_stopping = EarlyStopping(patience = 15) # 조기종료 콜백함수 정의

# 데이터 개수 / batch_size
steps_per_epoch = 54332 / batch_size
val_steps = 13585 / batch_size

set_trainable = False
for layer in pre_trained_model.layers:
    if layer.name == 'conv2d_386':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-7), loss='categorical_crossentropy',
                metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=val_steps,
                    callbacks=[early_stopping],
                    verbose=1
                    )

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_dir = 'C:/Users/s_csmscox/jupyterSave/eye_blink/test'

batch_size = 50
img_size = 32

test_datagen = ImageDataGenerator(rescale=1/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size,img_size),                      
    batch_size=batch_size,     
    class_mode='categorical'                    
)

In [ ]:
# 모델 평가

steps = 16981 / batch_size

print("-- Evaluate --")
scores = model.evaluate(test_generator, steps=steps)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))